<a href="https://colab.research.google.com/github/ChibuzoKelechi/GoogleColab_notebooks/blob/main/sentiment_classifier_tf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import datetime
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from tensorflow import keras as tfkeras
from keras import layers, losses
from keras.models import load_model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [23]:
reddit_data = '/content/drive/MyDrive/Datasets/Twitter_reddit SA data/Reddit_Data.csv'
# reddit_data = 'https://drive.google.com/file/d/1ludyoukT1kH5YkEiIkuA5WmCjsMig4T4/view?usp=drive_link'

dataset = pd.read_csv(reddit_data)

comments = dataset['clean_comment']
categories = dataset['category']

sentences = []
labels = []

for sentence in comments:
  # sentences.append(str(dataset['clean_comment'].tolist()))
  sentences.append(str(sentence))

for category in categories:
  labels.append(category)

train_size = 28000
vocab_size = 48244
embedding_dim = 50
max_length = 100
trunc_type='post'
pad_type='post'
training_size = 20000

train_text = sentences[0:train_size]
test_text = sentences[train_size:]

train_labels = labels[0:train_size]
test_labels = labels[train_size:]

In [28]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token='<OOV>')

tokenizer.fit_on_texts(train_text)
word_index = tokenizer.word_index


train_sequence = tokenizer.texts_to_sequences(train_text)
train_padded = pad_sequences(train_sequence, padding='post', maxlen=100, truncating=trunc_type)

test_sequence = tokenizer.texts_to_sequences(test_text)
test_padded = pad_sequences(test_sequence, padding='post', maxlen=100, truncating=trunc_type)

train_padded = np.array(train_padded)
train_labels = np.array(train_labels)
test_padded = np.array(test_padded)
test_labels = np.array(test_labels)

train_labels = train_labels.reshape(-1, 1)
test_labels = test_labels.reshape(-1, 1)

test_labels = test_labels[:len(test_padded)]

dataset_shape = dataset.shape

train_labels = to_categorical(train_labels, num_classes=3)
test_labels = to_categorical(test_labels, num_classes=3)


In [29]:
early_stop = tfkeras.callbacks.EarlyStopping(
    min_delta=0.001,
    patience=10,
    restore_best_weights=True
)

In [30]:
text_model = tfkeras.Sequential([
    layers.Embedding(vocab_size, embedding_dim, input_length=100),
    layers.GlobalAveragePooling1D(),
    layers.Dropout(0.3),
    layers.BatchNormalization(),
    layers.Dense(10, activation='relu'),
    layers.Dense(3, activation='softmax')
])


text_model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [31]:
history = text_model.fit(train_padded, train_labels,
    epochs=30,
    validation_data=(test_padded, test_labels),
    verbose=1
    # callbacks=[early_stop]
)

Epoch 1/30
875/875 [==============================] - 31s 34ms/step - loss: 0.8026 - accuracy: 0.6373 - val_loss: 0.6202 - val_accuracy: 0.7784
Epoch 2/30
875/875 [==============================] - 28s 32ms/step - loss: 0.4285 - accuracy: 0.8416 - val_loss: 0.4222 - val_accuracy: 0.8544
Epoch 3/30
875/875 [==============================] - 28s 32ms/step - loss: 0.2838 - accuracy: 0.8998 - val_loss: 0.5175 - val_accuracy: 0.8086
Epoch 4/30
875/875 [==============================] - 28s 32ms/step - loss: 0.2334 - accuracy: 0.9180 - val_loss: 0.5068 - val_accuracy: 0.8300
Epoch 5/30
875/875 [==============================] - 29s 33ms/step - loss: 0.1876 - accuracy: 0.9332 - val_loss: 0.6047 - val_accuracy: 0.8267
Epoch 6/30
875/875 [==============================] - 28s 32ms/step - loss: 0.1682 - accuracy: 0.9406 - val_loss: 0.5008 - val_accuracy: 0.8531
Epoch 7/30
875/875 [==============================] - 28s 32ms/step - loss: 0.1590 - accuracy: 0.9444 - val_loss: 0.5205 - val_accuracy: